<a href="https://colab.research.google.com/github/jhk0530/spider/blob/master/71_%EC%9C%A0%EC%82%AC_%EC%9D%B4%EB%AF%B8%EC%A7%80_%EA%B2%80%EC%B6%9C%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 이미지 인식 - Average Hash

`Average Hash`란, 이미지를 비교 가능한 해시 값으로 나타내는것, 해시함수 `MD5`, `SHA256`등을 이용하여 변환.

그러나 이미지가 비슷한지 검출할때는 사용하면 안됨 이유는 이미지데이터에

- 해상도 조절
- 색 보정
- PNG 등 형식 변경등으로 인해

완전한 바이너리를 찾는것이 어려움.

이로 인해 이미지가 조금 밝게 혹은 어둡게처럼 약간 변경후에도 이미지를 비교 할 경우 `Average Hash`를 사용함

예시 프로세스 과정

- 이미지를 8x8 사이즈로 축소 시킴
- 색을 그레이스케일로 변환
- 이미지의 각 픽셀의 평균을 계산하여 평균보다 크면 1, 작으면 0을 반환 

이렇게 하면 이미지마다 64비트의 해시 값을 구할 수 있음.

사용하는 library는 `pillow`






In [1]:
!pip3 install Pillow



In [2]:
# 파일 다운로드

import urllib.request

url = 'https://yotsuya.hotelkeihan.co.jp/wp-content/uploads/sites/394/2019/02/Tokyo-Tower-500x250.jpg'

savename  ='tower.jpg'
urllib.request.urlretrieve(url, savename)

print('저장되었습니다...!')


저장되었습니다...!


In [3]:
from PIL import Image
import numpy as np

def average_hash (fname, size = 16):
  img = Image.open(fname)
  img = img.convert('L') # 그레이 스케일로 변환
  img = img.resize((size,size), Image.ANTIALIAS) # 리사이즈
  
  pixel_data = img.getdata() # 픽셀값
  pixels = np.array(pixel_data) # numpy로 치환
  pixels = pixels.reshape((size, size)) # 2 차원으로 변환

  avg = pixels.mean()
  diff = 1* (pixels>avg)

  return diff

def np2hash(ahash):
  bhash  =[]
  
  for nl in ahash.tolist():
    s1 = [str(i) for i in nl]
    s2 = ''.join(s1)
    i = int(s2,2)
    bhash.append('%04x' % i)
  
  return "".join(bhash)

ahash = average_hash('tower.jpg')
print(ahash)
print(np2hash(ahash))


[[0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 1 1 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1]
 [0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 1]
 [0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 1]
 [0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 1]
 [0 0 0 0 1 1 1 1 0 0 0 0 1 1 1 1]]
0380038007c00780038007c006400f600f210f01060305070d0701170d1f0f0f


In [4]:
# CALTECH Data URL = http://www.vision.caltech.edu/Image_datasets/Caltech101/Caltech101.html

# http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz


import os

os.getcwd() # /content

#import urllib.request

#url = 'http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz'

#savename  ='101_ObjectCategories.tar.gz'
#urllib.request.urlretrieve(url, savename)

# print('저장되었습니다...!')




'/content'

데이터 저장하려는데 디렉토리 구조가 안보임 ㅅㄱ

파일 `contents` 잘못 옮겨서 터진거였다
초기화 하려면, 노트 새로 만들기 하지말고 그냥 런타임 초기화 하면 됨 ~

데이터는 다 쓰는게 아닌 yinyang 음양만 씀




In [9]:
from PIL import Image
import numpy as np
import os, re

search_dir = './'
cache_dir = './cache_avhash'

# 캐시 디렉토리 생성
if not os.path.exists(cache_dir):
  os.mkdir(cache_dir)

def average_hash(fname, size = 16):
  fname2 = fname[len(search_dir):]

  cache_file = cache_dir + '/' + fname2.replace('/', '_')+ ".csv"
  
  if not os.path.exists(cache_file):# 캐시 없는 경우, 생성
    img = Image.open(fname)
    img = img.convert('L').resize((size,size), Image.ANTIALIAS)
    pixels = np.array(img.getdata()).reshape((size,size))
    avg = pixels.mean()
    px = 1*(pixels>avg)
    np.savetxt(cache_file, px, fmt = '%.0f', delimiter = ',')
  else : 
    px = np.loadtxt(cache_file, delimiter=',')

  return px


def hamming_dist(a,b):
  aa = a.reshape(1,-1)
  ab = b.reshape(1,-1)
  dist = (aa!=ab).sum()
  return dist

def enum_all_files(path):
  for root,dirs, files in os.walk(path):
    for f in files:
      fname = os.path.join(root,f)
      if re.search(r'\.(jpg|jpeg|png)$', fname):
        yield fname

def find_image(fname, rate):
  src = average_hash(fname)
  for fname in enum_all_files(search_dir):
    dst = average_hash(fname)
    diff_r = hamming_dist(src,dst) / 256
    if diff_r < rate : 
      yield(diff_r, fname)

srcfile = search_dir+'image_0016.jpg'
html = ''
sim = list(find_image(srcfile,0.25))
sim = sorted(sim, key = lambda x:x[0])
for r,f in sim:
  print(r,'>',f)
  s = "<div style='float:left'><h3>[차이 : " + str(r) + ' - ' + os.path.basename(f) + ']</h3>' + '<p><a href = "' + f + '"> <img src="' +f+ '" width = 400>'+ '</a></p></div>'
  html += s

html = """<html>
<head><meta charset = 'utf8'></head>
<body>
  <h3>원래 이미지 <h3>
  <p><img src='{0}'> width = 400></p>{1}
</body>
</html>""".format(srcfile,html)
with open('./avhash-search-output.html', 'w', encoding = 'utf-8') as f :
  f.write(html)

print("ok")



0.0 > ./image_0016.jpg
0.16015625 > ./image_0028.jpg
0.16796875 > ./image_0030.jpg
0.19140625 > ./image_0021.jpg
0.21875 > ./image_0008.jpg
0.23046875 > ./image_0044.jpg
ok


html 리포트가 잘 안나오긴 하지만 어쨌던, 이미지 비교 - 분석은 잘 됨.